### 사용자 정의 모델 클래스
- 부모 클래스 : nn.Module
- 필수 오버라이딩 
    *  __init__() : 모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

In [75]:
## 모듈 로딩
import torch                     # 텐서 관련 모듈
import torch.nn as nn            # 인공신경망 모듈
import torch.nn.functional as F  # 인공신경망 관련 함수들 모듈(손실함수, 활성화함수 등등)
import torch.optim as optim      # 최적화 관련 모듈( 가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary    # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import * # *=> regression안에 있는 애들 다 들고옴
# 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *
# 분류 성능 지표 관련 모듈


In [76]:
# 랜덤 고정
torch.manual_seed(7)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 : 피쳐수 고정
    * 출력층 - 출력 : 타겟수 고정
    * 은닉층 - 고정

In [77]:
# 모델 설계
# 데이터 셋 : 피쳐 4개, 타겟 1개, 회귀
# 입력층 : 입력  4개   출력  20개    AF  ReLU
# 은닉층 : 입력  20개  출력  100개   AF  ReLU
# 출력층 : 입력  100개 출력  1개     AF  X,                                      Sigmoid & Softmax(다중)
#                                                                              (분류에서는 확률로 변환해서 줌)

In [78]:
# 클래스 상속
class MyModel(nn.Module):
    # __ooo__ : 매직 메서드 : 파이썬 시스템에 의해 호출되는 메서드
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self):
        # 부모 클래서 먼저 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(4,20)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20,100)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100,1)   # W 100 + b 1 => 1P , 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self,x):
        print('calling forward()')
        y = self.input_layer(x) # 1개 퍼셉트론 : y = x1w1 + x2w2 + x3w3 + x4w4 + b
        y = F.relu(y) # 엑티베이션 함수 거쳐서 결과값 냄
        #   0 <= y  -----> 죽은 relu ==> leakyReLu

        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x19w19 + x20w20 + b
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x99w99 + x100w100 + b

       


In [79]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module):
    # __ooo__ : 매직 메서드 : 파이썬 시스템에 의해 호출되는 메서드
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self,in_feature):
        # 부모 클래서 먼저 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,20)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20,100)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100,1)   # W 100 + b 1 => 1P , 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self,x):
        print('calling forward()')
        y = self.input_layer(x) # 1개 퍼셉트론 : y = x1w1 + x2w2 + x3w3 + x4w4 + b
        y = F.relu(y) # 엑티베이션 함수 거쳐서 결과값 냄
        #   0 <= y  -----> 죽은 relu ==> leakyReLu

        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x19w19 + x20w20 + b
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x99w99 + x100w100 + b


In [80]:
# 입력 피쳐 수, 은닉츠 퍼셉트론 수 동적인 모델
class MyModel3(nn.Module):
    # __ooo__ : 매직 메서드 : 파이썬 시스템에 의해 호출되는 메서드
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self,in_feature,in_out,hid_out):
        # 부모 클래서 먼저 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,in_out)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(in_out,hid_out)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(hid_out,1)   # W 100 + b 1 => 1P , 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self,x):
        print('calling forward()')
        y = self.input_layer(x) # 1개 퍼셉트론 : y = x1w1 + x2w2 + x3w3 + x4w4 + b
        y = F.relu(y) # 엑티베이션 함수 거쳐서 결과값 냄
        #   0 <= y  -----> 죽은 relu ==> leakyReLu

        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x19w19 + x20w20 + b
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x99w99 + x100w100 + b


In [94]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    # __ooo__ : 매직 메서드 : 파이썬 시스템에 의해 호출되는 메서드
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self,in_feature,in_out,hid_out,hid_cnt):
        # 부모 클래서 먼저 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,in_out)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        for i in range(hid_cnt):
            
            self.hidden_layer = nn.Linear(in_out,hid_out)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
            
        self.output_layer = nn.Linear(hid_out,1)   # W 100 + b 1 => 1P , 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self,x):
        print('calling forward()')
        y = self.input_layer(x) # 1개 퍼셉트론 : y = x1w1 + x2w2 + x3w3 + x4w4 + b
        y = F.relu(y) # 엑티베이션 함수 거쳐서 결과값 냄
        #   0 <= y  -----> 죽은 relu ==> leakyReLu

        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x19w19 + x20w20 + b
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 ****+ x99w99 + x100w100 + b


In [95]:
## 모델 인스턴스 생성
m1=MyModel4(3,50,30,5)
m1

MyModel4(
  (input_layer): Linear(in_features=3, out_features=50, bias=True)
  (hidden_layer): Linear(in_features=50, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=1, bias=True)
)

In [82]:
## 모델 파라미터 즉, W와 b 확인
for m in m1.named_parameters() : print(m)

('input_layer.weight', Parameter containing:
tensor([[ 0.0403, -0.3478,  0.1838],
        [ 0.1812, -0.3086, -0.0865],
        [-0.3382,  0.1498, -0.1555],
        [ 0.4056,  0.4099,  0.0588],
        [-0.2461, -0.3391, -0.0634],
        [-0.1625,  0.2545, -0.4930],
        [ 0.5426, -0.4529,  0.4421],
        [-0.1003,  0.2970,  0.2250],
        [ 0.0242,  0.1077,  0.4384],
        [ 0.1485,  0.3063, -0.4466],
        [ 0.4110,  0.1987,  0.1462],
        [ 0.0798,  0.2814,  0.5303],
        [-0.1285, -0.3216, -0.1453],
        [-0.3519,  0.2777, -0.2853],
        [-0.3081,  0.4982,  0.5283],
        [ 0.0664, -0.1000, -0.0745],
        [ 0.2735, -0.5391, -0.4718],
        [ 0.4612,  0.5700, -0.0343],
        [-0.4562,  0.0158, -0.2686],
        [-0.0011,  0.2826,  0.2556],
        [-0.0677,  0.0636,  0.1571],
        [-0.4525, -0.1957,  0.0226],
        [-0.3294, -0.2521,  0.1939],
        [ 0.3323,  0.0081, -0.2246],
        [ 0.3161, -0.3165, -0.1355],
        [ 0.5447,  0.1063,  0.

In [83]:
# 학습 진행 ==> 모델 인스턴스명(데이터)
# 임의의 데이터
dataTS = torch.FloatTensor([[1,3,5],[2,4,6]])
targetTS = torch.FloatTensor([[4],[5]])

# 학습
pre_y = m1(dataTS)

print(pre_y)

calling forward()
tensor([[-0.4332],
        [-0.4814]], grad_fn=<AddmmBackward0>)
